In [88]:
import pandas as pd
import ast
import json

from lark import Lark, UnexpectedInput

In [89]:
%run "../read_and_write_docs.py"

In [170]:
# g_drive_loc = "/Users/user/Library/CloudStorage/GoogleDrive-benjcross1995@gmail.com/My Drive/"
# batch_complete_loc = f"{g_drive_loc}datasets/blogger/batch_complete/"
batch_complete_loc = "../../../../datasets/blogger/batch_complete/"
batch_save_loc = "../../../../datasets/blogger/batch_4o/"
batch_fail_loc = "../../../../datasets/blogger/batch_4o_fail/"

In [171]:
grammar_loc = "../../grammars/list_enf_format.lark"

In [172]:
def validate_and_parse_response(df, grammar_file):
    # Load the grammar
    with open(grammar_file, 'r') as file:
        grammar = file.read()
    
    parser = Lark(grammar, start='root', parser='earley')

In [173]:
# Function to extract the desired parts
def split_custon_id(custon_id):
    parts = custon_id.split('_')
    doc_id = parts[1]
    chunk_id = parts[3]
    repetition = parts[4]
    return pd.Series([doc_id, chunk_id, repetition])

In [174]:
def parse_response(response_str):
    # Convert the JSON string to a Python dictionary
    response_dict = json.loads(response_str)
    
    # Extract the 'original' sentence
    original_sentence = response_dict.get('original', '')
    
    # Extract other keys and add them to the list with 'repetition_i' format
    rephrased = []
    for key, value in response_dict.items():
        if key != 'original':
            rephrased.append(value)
    
    return original_sentence, rephrased

def process_dataframe(df):

    df[['doc_id', 'chunk_id', 'repetition']] = df['custom_id'].apply(split_custon_id)
    
    # Apply the parse_response function to each row of the dataframe
    df['original_sentence'], df['rephrased'] = zip(*df['response'].apply(parse_response))

    df = df[['doc_id', 'chunk_id', 'original_sentence', 'rephrased']]
    
    return df

In [175]:
def combine_and_unique(group):
    # Combine all lists in the 'rephrased' column into one list
    combined_list = sum(group['rephrased'], [])
    
    # Remove duplicates by converting to a set and back to a list
    unique_list = list(set(combined_list))
    
    # Return a DataFrame where each unique rephrased sentence is a new row
    return pd.DataFrame({
        'doc_id': group['doc_id'].iloc[0],
        'chunk_id': group['chunk_id'].iloc[0],
        'original': group['original_sentence'].iloc[0],
        'rephrased': unique_list
    })

def process_rephrased_sentences(df):
    # Group by 'doc_id' and 'chunk_id' and apply the combine_and_unique function
    result_df = df.groupby(['doc_id', 'chunk_id']).apply(combine_and_unique).reset_index(drop=True)
    return result_df

In [176]:
def process_file_list(files, base_read_loc, base_write_loc, base_fail_loc):

    for file in files:
        
        response = read_jsonl_file(f"{base_read_loc}{file}")
        print(f"{file} Loaded")
        
        processed_dataframe = process_dataframe(response)
        print(f"{file} Processed")

        final_df = process_rephrased_sentences(processed_dataframe)
        print(f"{file} List Split")
        
        save_loc = f"{base_write_loc}{file.replace('batch', 'doc')}"

        save_as_jsonl(final_df, save_loc)

In [177]:
import os
import shutil

def process_file_list(files, base_read_loc, base_write_loc, base_fail_loc):
    for file in files:
        try:
            # Step 1: Read the JSONL file
            response = read_jsonl_file(f"{base_read_loc}{file}")
            print(f"{file} Loaded")
            
            # Step 2: Process the DataFrame
            processed_dataframe = process_dataframe(response)
            print(f"{file} Processed")

            # Step 3: Split the list into separate rows
            final_df = process_rephrased_sentences(processed_dataframe)
            print(f"{file} List Split")
            
            # Step 4: Save the processed DataFrame to a new location
            save_loc = f"{base_write_loc}{file.replace('batch', 'doc')}"
            save_as_jsonl(final_df, save_loc)
            print(f"{file} Saved to {save_loc}")
            
        except Exception as e:
            # Step 5: If any error occurs, save the file to the fail location
            fail_loc = f"{base_fail_loc}{file}"
            print(f"Error processing {file}: {e}")
            shutil.move(f"{base_read_loc}{file}", fail_loc)
            print(f"{file} Moved to {fail_loc}")
            continue  # Move to the next file in case of an error
        
        # Step 6: Remove the original file from the read location if processing succeeds
        os.remove(f"{base_read_loc}{file}")
        print(f"{file} Removed from {base_read_loc}")

In [178]:
files_to_be_processed = [
    f for f in os.listdir(batch_complete_loc)
    if os.path.isfile(os.path.join(batch_complete_loc, f)) and f.endswith('.jsonl')
]

files_processed = [
    f for f in os.listdir(batch_save_loc)
    if os.path.isfile(os.path.join(batch_save_loc, f)) and f.endswith('.jsonl')
]

# Extracting the numeric part from files_processed and transforming it back to batch form
processed_batches = {f"batch_{file.split('_')[1]}" for file in files_processed}
processed_ids = {file.split('_')[1].split('.')[0] for file in files_processed}

# Finding files in files_to_be_processed that are not in processed_batches
files_not_processed = [file for file in files_to_be_processed if file.split('_')[1].split('.')[0] not in processed_ids]

In [179]:
len(files_not_processed)

100

In [180]:
process_file_list(files_not_processed, batch_complete_loc, batch_save_loc, batch_fail_loc)

batch_617540.jsonl Loaded
batch_617540.jsonl Processed
batch_617540.jsonl List Split


/var/folders/xx/hy496x3x5sn4hy9gy1fk19lw0000gp/T/ipykernel_59736/2181975156.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result_df = df.groupby(['doc_id', 'chunk_id']).apply(combine_and_unique).reset_index(drop=True)


batch_617540.jsonl Saved to ../../../../datasets/blogger/batch_4o/doc_617540.jsonl
batch_617540.jsonl Removed from ../../../../datasets/blogger/batch_complete/
batch_579188.jsonl Loaded
batch_579188.jsonl Processed
batch_579188.jsonl List Split
batch_579188.jsonl Saved to ../../../../datasets/blogger/batch_4o/doc_579188.jsonl
batch_579188.jsonl Removed from ../../../../datasets/blogger/batch_complete/
batch_448015.jsonl Loaded
batch_448015.jsonl Processed


/var/folders/xx/hy496x3x5sn4hy9gy1fk19lw0000gp/T/ipykernel_59736/2181975156.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result_df = df.groupby(['doc_id', 'chunk_id']).apply(combine_and_unique).reset_index(drop=True)
/var/folders/xx/hy496x3x5sn4hy9gy1fk19lw0000gp/T/ipykernel_59736/2181975156.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result_df = df.groupby(['doc_id', 'chunk_id']).apply(com

batch_448015.jsonl List Split
batch_448015.jsonl Saved to ../../../../datasets/blogger/batch_4o/doc_448015.jsonl
batch_448015.jsonl Removed from ../../../../datasets/blogger/batch_complete/
batch_671397.jsonl Loaded
batch_671397.jsonl Processed
batch_671397.jsonl List Split


...

batch_551652.jsonl Saved to ../../../../datasets/blogger/batch_4o/doc_551652.jsonl
batch_551652.jsonl Removed from ../../../../datasets/blogger/batch_complete/
batch_491687.jsonl Loaded
batch_491687.jsonl Processed
batch_491687.jsonl List Split


/var/folders/xx/hy496x3x5sn4hy9gy1fk19lw0000gp/T/ipykernel_59736/2181975156.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result_df = df.groupby(['doc_id', 'chunk_id']).apply(combine_and_unique).reset_index(drop=True)


batch_491687.jsonl Saved to ../../../../datasets/blogger/batch_4o/doc_491687.jsonl
batch_491687.jsonl Removed from ../../../../datasets/blogger/batch_complete/
batch_463289.jsonl Loaded
batch_463289.jsonl Processed
batch_463289.jsonl List Split
batch_463289.jsonl Saved to ../../../../datasets/blogger/batch_4o/doc_463289.jsonl
batch_463289.jsonl Removed from ../../../../datasets/blogger/batch_complete/
batch_171105.jsonl Loaded
batch_171105.jsonl Processed
batch_171105.jsonl List Split


/var/folders/xx/hy496x3x5sn4hy9gy1fk19lw0000gp/T/ipykernel_59736/2181975156.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result_df = df.groupby(['doc_id', 'chunk_id']).apply(combine_and_unique).reset_index(drop=True)
/var/folders/xx/hy496x3x5sn4hy9gy1fk19lw0000gp/T/ipykernel_59736/2181975156.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result_df = df.groupby(['doc_id', 'chunk_id']).apply(com

batch_171105.jsonl Saved to ../../../../datasets/blogger/batch_4o/doc_171105.jsonl
batch_171105.jsonl Removed from ../../../../datasets/blogger/batch_complete/


In [166]:
fail_test =read_jsonl_file(f"{batch_complete_loc}batch_144356.jsonl")
processed_dataframe = process_dataframe(fail_test)
final_df = process_rephrased_sentences(processed_dataframe)

JSONDecodeError: Unterminated string starting at: line 21 column 20 (char 23204)

In [169]:
fail_test

,id,custom_id,response,doc_id,chunk_id,repetition
0,batch_req_iwJgunTo81E2QukR0O3D8s0X,doc_144356_chunk_1_1,"{""original"": ""Almost the end of July.&nbsp; Ho...",144356,1,1
1,batch_req_koOWaGvqW0AZCglgHlo6eYuX,doc_144356_chunk_1_2,"{""original"": ""Almost the end of July. How is y...",144356,1,2
2,batch_req_1O2H6cY6YRsBEgm8is5IFz1T,doc_144356_chunk_1_3,"{""original"": ""Almost the end of July.&nbsp; Ho...",144356,1,3
3,batch_req_yZOdBE93nNZWRBhDRyt3CTEn,doc_144356_chunk_1_4,"{""original"": ""Almost the end of July. How is y...",144356,1,4
4,batch_req_VSe2v2S3Uay1xbhZ5PQRHsZK,doc_144356_chunk_1_5,"{""original"": ""Almost the end of July. How is y...",144356,1,5
5,batch_req_HBUp6qGcAlzxNsStFhJ3GBKM,doc_144356_chunk_1_6,"{""original"": ""Almost the end of July. How is y...",144356,1,6
6,batch_req_9mU0h5BHyqXpIUUcRO5fRrSl,doc_144356_chunk_1_7,"{""original"": ""Almost the end of July. How is y...",144356,1,7
7,batch_req_YdgXrNdRcMia0ZcPmjrTtexP,doc_144356_chunk_1_8,"{""original"": ""Almost the end of July. How is y...",144356,1,8
8,batch_req_tTzKiCFDvn4Xle2jAweFI0Mb,doc_144356_chunk_1_9,"{""original"": ""Almost the end of July.&nbsp; Ho...",144356,1,9
9,batch_req_rZsjSdN4BpyHC9tsTtOzhPup,doc_144356_chunk_1_10,"{""original"": ""Almost the end of July. How is y...",144356,1,10


In [168]:
for index, row in fail_test.iterrows():
    response_str = row['response']
    try:
        response_json = json.loads(response_str)
    except json.JSONDecodeError:
        print(f"Error parsing JSON in row {index}: {response_str}")

Error parsing JSON in row 1: {"original": "Almost the end of July. How is your summer going? Yesterday I visited some Blogs on the Internet and picked up a nasty adware or spyware virus, which has disabled my Internet browser, closes open windows automatically, opens up pop-up advertising windows indiscriminately, and seems to keep reinstalling itself, despite my best attempts to delete any foreign files that I find in my operating system. Mind you, I am such an inept technophobe that I couldn’t differentiate a malicious virus from a harmless bug. None the less, as I am not the administrator of my computer at work, I have to wait until he comes back on Monday to access my computer remote, and reboot it after, hopefully, isolating and decontaminating my hard drive. What a nuisance. I was doubly surprised to find out that my networked PC did not even have any anti-virus software on it, thereby making it a perfect roaming target for this adware or spyware to attach itself to my computer, 

In [161]:
final_df

,doc_id,chunk_id,original,rephrased
0,617540,1,It has been suggested that I share some storie...,I have been recommended to talk about some sto...
1,617540,1,It has been suggested that I share some storie...,I've been asked to recount stories about my st...
2,617540,1,It has been suggested that I share some storie...,There have been recommendations for me to reco...
3,617540,1,It has been suggested that I share some storie...,People have suggested that I share a few tales...
4,617540,1,It has been suggested that I share some storie...,It has been proposed that I tell some stories ...
...,...,...,...,...
7943,617540,9,they always have cash and insist on paying for...,They consistently carry cash and make it a poi...
7944,617540,9,they always have cash and insist on paying for...,They always seem to have cash available and in...
7945,617540,9,they always have cash and insist on paying for...,They are always seen with cash and are adamant...
7946,617540,9,they always have cash and insist on paying for...,They typically carry cash and insist on handli...


In [156]:
save_loc = f"{batch_save_loc}batch_402496.jsonl"
save_as_jsonl(final_df, save_loc)

In [33]:
grammar = r"""
start: "[" items "]"

items: item ("," item)*

item: ESCAPED_STRING

ESCAPED_STRING: "\"" /[a-zA-Z0-9.,!?;:’\-()' ]+/ "\""

%ignore " "
%ignore "\n"
"""

parser = Lark(grammar, parser='earley')

In [36]:
grammar = r"""
start: "[" items "]"

items: item ("," item)*

item: ESCAPED_STRING

ESCAPED_STRING: "\"" /[^"]+/ "\""

%ignore " "
%ignore "\n"
"""

parser = Lark(grammar, parser='earley')

In [37]:
def validate_and_parse_response(df, parser):
    # # Load the grammar
    # with open(grammar_file, 'r') as file:
    #     grammar = file.read()
    
    # parser = Lark(grammar, start='start', parser='earley')

    # Function to parse each response
    def parse_response(response):
        try:
            # Check if the response is a string, if so, parse it
            if isinstance(response, str):
                # Validate the response using the grammar
                parser.parse(response)
                # Convert the string to an actual list
                return ast.literal_eval(response)
            else:
                return response
        except (UnexpectedInput, ValueError, SyntaxError) as e:
            print(f"Invalid response format: {response} - Error: {e}")
            return response  # Return the original string if parsing fails

    # Loop through each row and validate/parse the 'response' column
    for index, row in df.iterrows():
        df.at[index, 'response'] = parse_response(row['response'])
    
    return df

In [38]:
test_converted = validate_and_parse_response(test, parser)

Invalid response format: ["It presents itself as more of a supportive scenario rather than a case of earning a hefty sum to have an unpleasant individual with a waste bag beside him perform an unsavory act on you.", 
 "It creates the impression of being a more charitable context rather than a lucrative opportunity to have a repulsive man with a bodily waste bag engage in an indecent act with you.", 
 "It feels more like a scenario where assistance is offered rather than a situation where you get paid a significant amount to have a disgusting man with a sanitary bag next to him satisfy your lower regions.", 
 "This makes it seem more like a situation where you're lending a hand instead of one where you receive substantial compensation to endure the presence of a foul person carrying a waste pouch licking you.", 
 "It comes across as a more altruistic arrangement than one that involves receiving a lot of money for enduring the advances of a nasty guy with a defecation bag beside him.", 


In [15]:
test_converted.iloc[0,2]

'["It has been proposed that I share a few tales about my stripper friends.", "Some have recommended that I tell some stories involving my stripper acquaintances.", "There has been a suggestion for me to relay stories of my stripper buddies.", "I’ve been advised to share anecdotes about my stripper pals.", "It has been hinted that I should narrate some experiences with my stripper friends.", "People have suggested that I recount some stories related to my stripper companions.", "Someone recommended that I share tales featuring my stripper friends.", "There has been an idea floated that I should talk about my stripper pals.", "It has been put forward that I narrate some stories about my stripper friends.", "I’ve received suggestions to tell some stories regarding my stripper friends.", "Some individuals have suggested that I share stories of my stripper buddies.", "I\'ve been encouraged to discuss my experiences with my stripper pals.", "It has been communicated that I might share some 

In [16]:
test_converted

,id,custom_id,response,doc_id,chunk_id,repetition
0,batch_req_SGhzjs2FC09ZbXkFCIFVKXAE,doc_617540_chunk_1_1,"[""It has been proposed that I share a few tale...",617540,1,1
1,batch_req_rzSSPUOj9hzG1RRSNSet8rEy,doc_617540_chunk_1_2,"[""It has been recommended that I share a few t...",617540,1,2
2,batch_req_m0hECxM0s1dGDMg7BUyybsxr,doc_617540_chunk_1_3,"[""It has been proposed that I recount some tal...",617540,1,3
3,batch_req_fNvwAF3S3oXC73x2NdNnzhmb,doc_617540_chunk_1_4,"[""It has been recommended that I share a few t...",617540,1,4
4,batch_req_4tzMUFTOq0L52SqnzAKehBVp,doc_617540_chunk_1_5,"[""It has been recommended that I tell some tal...",617540,1,5
...,...,...,...,...,...,...
465,batch_req_ykOKTuWBaylBo3jUIwPh2zMp,doc_617540_chunk_47_6,"[""But I have a feeling that someone made a mov...",617540,47,6
466,batch_req_VO6mIoTEcEcxWWee9Wvxg6KZ,doc_617540_chunk_47_7,"[""But I have a feeling that someone made a mov...",617540,47,7
467,batch_req_B8U6pJ7ToLMa3H39Srserxs9,doc_617540_chunk_47_8,"[""But I have a feeling that someone flirted wi...",617540,47,8
468,batch_req_w50FDCIKy07GW6fOILlCuQpy,doc_617540_chunk_47_9,"[""But I have a feeling that someone flirted wi...",617540,47,9


In [31]:
from lark import Lark, Transformer

grammar = r"""
start: "[" items "]"

items: item ("," item)*

item: string

string: "\"" word (ws+ word)* "\"" 

word: /[a-zA-Z0-9.,!?;:’\-()']+/

ws: " "

%ignore ws
%ignore "\n"
"""

parser = Lark(grammar, parser='earley')

class ListTransformer(Transformer):
    def start(self, items):
        return items[0]

    def items(self, items):
        return items

    def item(self, item):
        return item[0]

    def string(self, string):
        return "".join(string)

    def word(self, word):
        return str(word[0])

tree = parser.parse('["hello", "world", "example"]')
result = ListTransformer().transform(tree)
print(result)


GrammarError: Rules aren't allowed inside terminals (NonTerminal('ws') in __IGNORE_0)